In [45]:
import instructor
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import List
import os
from dotenv import load_dotenv

load_dotenv()

# Enables `response_model`
client = instructor.patch(OpenAI())

In [ ]:


class UserDetail(BaseModel):
    name: str
    age: int

user = client.chat.completions.create(
    model="gpt-3.5-turbo",
    response_model=UserDetail,
    messages=[
        {"role": "user", "content": "Extract Brian is 46 years old"},
    ]
)

assert isinstance(user, UserDetail)
assert user.name == "Brian"
assert user.age == 46

json_str = user.model_dump_json()
print(json_str)

In [ ]:

class Website(BaseModel):
    name: str
    url: str

class WebsiteExtraction(BaseModel):
    websites: List[Website] = Field(
        ...,
        description="Body of the answer, each website should be a separate object with a name and a url of the website",
    )

def get_websites(prompt) -> WebsiteExtraction:
    website_extraction: WebsiteExtraction = client.chat.completions.create(
        model="gpt-3.5-turbo",
        response_model=WebsiteExtraction,
        messages=[
        {
            "role": "system",
            "content": "You are an expert aws cloud solutions architect with 20 years experience. answer exactly what the question asks using the context.",
        },            
            {"role": "user", "content": prompt},
        ]
    )
    return '\n'.join([f"{website.name}, {website.url}" for website in website_extraction.websites])
    

print(get_websites("Extract the top 20 technology websites"), "\n")

print(get_websites("""
    Extract the top 20 AWS twitter accounts.
    Each twitter account contain a name and url.
    The twitter account should not be owned by AWS.
"""), "\n")

print(get_websites("""
    Extract the top 20 AWS websites.
    Each website should contain a name and url
    The website should not be owned by AWS .
    The website url should not have 'aws.amazon.com' as a substring
"""), "\n")


In [49]:
class File(BaseModel):
    """
    Correctly named file with contents.
    """

    file_name: str = Field(
        ..., description="The name of the file including the extension"
    )
    body: str = Field(..., description="Correct contents of a file")

    def save(self):
        with open(self.file_name, "w") as f:
            f.write(self.body)


class Program(BaseModel):
    """
    Set of files that represent a complete and correct program
    """

    files: List[File] = Field(..., description="List of files")


def develop(data: str) -> Program:
    return client.chat.completions.create(
        model="gpt-3.5-turbo-0613",
        temperature=0.1,
        response_model=Program,
        messages=[
            {
                "role": "system",
                "content": "You are a world class programming AI capable of writing correct python scripts and modules. You will name files correct, include __init__.py files and write correct python code with correct imports.",
            },
            {
                "role": "user",
                "content": data,
            },
        ],
        max_tokens=1000,
    )

def codegen(prompt, output_directory_path):
    debug_output = ''
    os.makedirs(output_directory_path, exist_ok=True)
    program = develop(prompt)
    for file in program.files:
        open(f"{output_directory_path}/{file.file_name}", "w").write(file.body)
    debug_output += file.file_name
    debug_output += "-"
    debug_output += file.body
    debug_output += "\n\n\n"
    return debug_output


# program = develop(
#         """
#         Create a fastapi app with a readme.md file and a main.py file with
#         some basic math functions. the datamodels should use pydantic and
#         the main.py should use fastapi. the readme.md should have a title
#         and a description. The readme should contain some helpful infromation
#         and a curl example"""
#     )

# program = develop(
#         """
#         Create a command line program with a readme.md file and a main.py file
#         that will take an absolute directory path as a command line argument and recursively print
#         the files and directories in the directory.  The  the datamodels should use pydantic and
#         the main.py should use argparse. the readme.md should have a title
#         and a description. The readme should contain some helpful infromation
#         and a command line usage example"""
#     )



program_debug_output = codegen(
        """
        Create a command line program with a readme.md file and a main.py file
        that will take an absolute directory path as a command line argument and recursively print
        the files and directories in the directory.  The  the datamodels should use pydantic and
        the main.py should use argparse. the readme.md should have a title
        and a description. The readme should contain some helpful infromation
        and a command line usage example""",
        "./codegen-recursive-directory-print"
    )


for file in program.files:
    print(file.file_name)
    print("-")
    print(file.body)
    print("\n\n\n")

readme.md
-
# Command Line Program

## Description

This command line program takes an absolute directory path as a command line argument and recursively prints the files and directories in the directory.

## Usage

```bash
python main.py /path/to/directory
```





main.py
-
import argparse
import os


def print_directory_contents(directory):
    for item in os.listdir(directory):
        item_path = os.path.join(directory, item)
        if os.path.isfile(item_path):
            print(item_path)
        elif os.path.isdir(item_path):
            print_directory_contents(item_path)


if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Print files and directories in a directory')
    parser.add_argument('directory', type=str, help='Absolute directory path')
    args = parser.parse_args()
    print_directory_contents(args.directory)





